In [ ]:
!pip install tpot

In [ ]:
from pandas import read_csv
path=r'/content/sample name - SPCF Jan 2022 - Revised.csv'
names=['SputterCu','SputterAl','Sputter(min)','Cu%']
import pandas as pd

features=X
labels=Y

dataframe=read_csv(path,names=names,skiprows=1,header=None)
data = dataframe.values
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train,Y_test= train_test_split(features,labels,test_size =0.2, random_state = 42)

tpot = TPOTRegressor(generations=50, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train, Y_train)
print(tpot.score(X_test, Y_test))
tpot.export('tpot tester2.py')





In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import FeatureAgglomeration
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from tpot.builtins import StackingEstimator
from xgboost import XGBRegressor
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('/content/sample name - SPCF Jan 2022 - Rev2.csv' dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=42)

# Average CV score on the training set was: -26.13086932867747
exported_pipeline = make_pipeline(
    FeatureAgglomeration(affinity="euclidean", linkage="average"),
    StackingEstimator(estimator=XGBRegressor(learning_rate=0.5, max_depth=8, min_child_weight=18, n_estimators=100, n_jobs=1, objective="reg:squarederror", subsample=0.1, verbosity=0)),
    StackingEstimator(estimator=SGDRegressor(alpha=0.01, eta0=1.0, fit_intercept=True, l1_ratio=1.0, learning_rate="invscaling", loss="huber", penalty="elasticnet", power_t=0.0)),
    LinearSVR(C=15.0, dual=False, epsilon=0.001, loss="squared_epsilon_insensitive", tol=0.0001)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


print(testing_target)
print(results)
